# Data preprocess
### Author: Yueqian Lin

## Step 1: Load data and get author list

In [1]:
# find the top 100 authors with high citation counts in the last 3 years

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

df= pd.read_csv("../result4.csv")

In [2]:
df.head()

,Title,Author Name,Author Id,1st Author Paper Count,1st Author Citation Count,Date,Abstract,Paper Reference Count,Paper Citation Count,Influential Citation Count,Venue,Fields Of Study
0,wav2vec 2.0: A Framework for Self-Supervised L...,"Alexei Baevski, Henry Zhou, Abdel-rahman Moham...","['51428394', '2110147709', '40360972', '2325985']",34.0,6941.0,2020-06-20,We show for the first time that learning power...,59,1633,314,Neural Information Processing Systems,"['Computer Science', 'Engineering']"
1,Conformer: Convolution-augmented Transformer f...,"Anmol Gulati, James Qin, C. Chiu, Niki Parmar,...","['4478284', '47901308', '145039780', '3877127'...",19.0,1571.0,2020-05-16,Recently Transformer and Convolution neural ne...,36,1099,214,Interspeech,"['Computer Science', 'Engineering']"
2,HiFi-GAN: Generative Adversarial Networks for ...,"Jungil Kong, Jaehyeon Kim, Jaekyoung Bae,","['1712238254', '94862572', '1703883959']",3.0,818.0,2020-10-12,Several recent work on speech synthesis have e...,27,505,116,Neural Information Processing Systems,"['Computer Science', 'Engineering']"
3,FastSpeech 2: Fast and High-Quality End-to-End...,"Yi Ren, Chenxu Hu, Xu Tan, Tao Qin, Sheng Zhao...","['1500435161', '2118956725', '48391466', '1438...",58.0,1974.0,2020-06-08,Advanced text to speech (TTS) models such as F...,54,514,105,International Conference on Learning Represent...,"['Computer Science', 'Engineering']"
4,The airborne lifetime of small speech droplets...,"V. Stadnytskyi, C. Bax, A. Bax, P. Anfinrud,","['3767476', '1633622443', '2074083973', '42414...",25.0,1303.0,2020-05-13,Speech droplets generated by asymptomatic carr...,20,790,39,Proceedings of the National Academy of Sciences,"['Medicine', 'Materials Science']"


In [74]:
# now get a list of top 100 paper's authors with high citation counts in the last 3 years

df1 = df.sort_values(by=['Paper Citation Count'], ascending=False).head(100)
authors = df1['Author Name'].tolist()
authors = [x for x in authors if str(x) != 'nan']
len(authors)

99

In [15]:
# Define a function to extract the author names from each entry
def extract_authors(entry):
    # Remove the trailing comma from the entry
    entry = entry.rstrip(',')
    # Split the entry into a list of individual author names
    author_list = entry.split(', ')
    # Remove any leading or trailing whitespace from the author names
    author_list = [author.strip() for author in author_list]
    # Return the list of author names
    return author_list

# Apply the function to each entry in the authors list
all_authors = []
for entry in authors:
    all_authors.extend(extract_authors(entry))
# Remove any duplicates from the list of authors
all_authors = list(set(all_authors))

In [19]:
# add ming li
all_authors.append('Ming Li')

In [69]:
ids = df1['Author Id'].str.strip("[]").str.replace("'", "").str.split(", ").explode().reset_index(drop=True)
ids = [x for x in ids if str(x) != 'nan']
all_ids = list(set(ids))
    

In [70]:
len(all_ids)

486

In [72]:
len(all_authors)

483

In [51]:
all_authors

['Barret Zoph',
 'Xiong Xiao',
 'M. F. Demirci',
 'T. Xiang',
 'H. Sak',
 'Muzammal Naseer',
 'Vedanuj Goswami',
 'Xulong Tan',
 'Douwe Kiela',
 'Jiawei Wang',
 'Furu Wei',
 'Joon Son Chung',
 'L. Hamner',
 'Daniel Haziza',
 'Shubo Lv',
 'Zion Mengesha',
 'C. Bax',
 'R. Mori',
 'Yong Zuo',
 'Henry Zhou',
 'Hyoukjun Kwon',
 'Pengcheng Guo',
 'Shimin Zhang',
 'Abdel-rahman Mohamed',
 'C. Talnikar',
 'P. Marsella',
 'Neil Zeghidour',
 'Munish Kumar',
 'Dominik Dellermann',
 'Kun Wei',
 'J. Leimeister',
 'Jiatao Gu',
 'Juhee Son',
 'A. Narayanan',
 'Andrew Zisserman',
 'E. Nicastri',
 'Sungwon Kim',
 'Hwidong Na',
 'Changhan Wang',
 'Ko-tik Lee',
 'P. White',
 'Hung-yi Lee',
 'M. Liao',
 'F. Khan',
 'Joe Nudell',
 'Nan Xue',
 'Xiaoshi Li',
 'Anuroop Sriram',
 'Simão Aznar Filho',
 'Emily Lake',
 'Shibo Wang',
 'Shijie Wu',
 'Shuo-yiin Chang',
 'Chris Biemann',
 'Zhengdong Zhang',
 'Z. Chen',
 'Yun Xiao',
 'Thomas Mandl',
 'Ju-Chieh Chou',
 'Amber Jordan',
 'Cem Subakan',
 'C. Cappa',
 'Car

"'1694491'"

## Step 2: Filter combinations from author list

In [20]:
comb = pd.read_csv("output_file.csv")
comb.head()

,Author 1,Author 2
0,Alexei Baevski,Henry Zhou
1,Alexei Baevski,Abdel-rahman Mohamed
2,Alexei Baevski,"Michael Auli,"
3,Henry Zhou,Abdel-rahman Mohamed
4,Henry Zhou,"Michael Auli,"


In [25]:
# filter according to the authors list
comb = comb[(comb['Author 1'].isin(all_authors) | comb['Author 2'].isin(all_authors))]
comb.to_csv("output_filtered.csv", index=False)

14665